In [1]:
import os
import sys
import time
import threading
from pathlib import Path
from time import sleep
import pyscan as ps
import ipywidgets as widgets
from IPython.display import display

Could not load Keysight SD1
Could not load Keysight SD1
pylablib not found, AttocubeANC350 not loaded
Basler Camera software not found, BaserCamera not loaded
Helios Camera not installed
msl not installed, Thorlabs BSC203 driver not loaded
seabreeze module not found, Ocean Optics not imported
Failed to load spinapi library.
spinapi is not installed, PulseBlaster driver not loaded.
Thorlabs Kinesis not found, ThorlabsBSC203 not loaded
Thorlabs Kinesis not found, ThorlabsBPC303 not loaded
Thorlabs Kinesis not found, ThorlabsMFF101 not loaded


In [2]:
instruments = ps.ItemAttribute()
instruments.ips120 = ps.new_instrument(gpib_address=25)
devices = ps.ItemAttribute()
devices.magnet = ps.OxfordIPS120(instruments.ips120, debug=False)

In [3]:
print(devices.magnet.print_state())

Heater on
At rest
activity: Hold
output_field = -4.3469
field_set_point = -5.0
field_rate = 0.049


## Testing magnet driver

In [4]:
# don't sweep anywhere while testing
devices.magnet.remote()
devices.magnet.hold()

### magnet limits

In [5]:
print(devices.magnet.field_limit)
print(devices.magnet.field_rate_limit)
print(devices.magnet.field_to_current)

8
0.2
0.22265764161026008


**Fix this safe current discrepency for the Triton magnet**

In [6]:
print(devices.magnet.safe_current_limit_positive)
print(devices.magnet.safe_current_limit_positive*devices.magnet.field_to_current)

40.42
8.999821873886713


### properties

In [7]:
print(devices.magnet.field_set_point)

-5.0


In [8]:
initial_value = devices.magnet.field_set_point
new_value = 0.111
devices.magnet.field_set_point = new_value
result = devices.magnet.field_set_point
assert result==new_value
devices.magnet.field_set_point = initial_value
result = devices.magnet.field_set_point
assert result==initial_value

In [9]:
print(devices.magnet.field_rate)

0.049


In [10]:
initial_value = devices.magnet.field_rate
new_value = 0.111
devices.magnet.field_rate = new_value
result = devices.magnet.field_rate
assert result==new_value
devices.magnet.field_rate = initial_value
result = devices.magnet.field_rate
assert result==initial_value

In [11]:
print(devices.magnet.current_set_point)

-22.456


In [12]:
initial_value = devices.magnet.current_set_point
new_value = 0.111
devices.magnet.current_set_point = new_value
result = devices.magnet.current_set_point
assert result==new_value
devices.magnet.current_set_point = initial_value
result = devices.magnet.current_set_point
assert result==initial_value

In [13]:
print(devices.magnet.current_rate)

0.22


In [14]:
initial_value = devices.magnet.current_rate
new_value = 0.22 # current can only be set to two digits in non-extended mode
devices.magnet.current_rate = new_value
result = devices.magnet.current_rate
assert result==new_value
devices.magnet.current_rate = initial_value
result = devices.magnet.current_rate
assert result==initial_value

### read-only properties

In [15]:
print(devices.magnet.output_current)

-19.523


In [16]:
print(devices.magnet.voltage)

-0.37


In [17]:
print(devices.magnet.measured_current)

-19.35


In [18]:
print(devices.magnet.output_field)

-4.3469


In [19]:
print(devices.magnet.software_voltage_limit)

2.5


In [20]:
print(devices.magnet.persistent_current)

0.0


In [21]:
print(devices.magnet.trip_field)

-32.803


In [22]:
print(devices.magnet.persistent_field)

0.0


In [23]:
print(devices.magnet.safe_current_limit_negative)

-40.42


In [24]:
print(devices.magnet.safe_current_limit_positive)

40.42


In [25]:
print(devices.magnet.lead_resistance)

18.7


In [26]:
print(devices.magnet.magnet_inductance)

51.7


In [29]:
print(devices.magnet.firmware_version)

IPS120-10  Version 3.07  (c) OXFORD 1996


In [30]:
print(devices.magnet.status_string)

X00A0C3H1M10P51


In [31]:
print(f"quench_status = {devices.magnet.quench_status}")

quench_status = False


In [32]:
print(f"heater_status = {devices.magnet.heater_status}")

heater_status = True


In [33]:
print(f"sweeping_status = {devices.magnet.sweeping_status}")

sweeping_status = False


In [34]:
print(f"persistent_status = {devices.magnet.persistent_status}")

persistent_status = False


In [35]:
print(f"remote_status = {devices.magnet.remote_status}")

remote_status = True


### write-only properties

In [36]:
devices.magnet.remote_control = "local_locked"

In [37]:
devices.magnet.remote_control = "remote_locked"

In [38]:
devices.magnet.remote_control = "local_unlocked"

In [39]:
devices.magnet.remote_control = "remote_unlocked"

In [40]:
devices.magnet.communications_protocol = "normal"

In [41]:
devices.magnet.current_rate = 0.222
print(devices.magnet.current_rate)

0.22


In [42]:
devices.magnet.communications_protocol = "extended"

In [43]:
devices.magnet.current_rate = 0.222
print(devices.magnet.current_rate)

0.222


In [44]:
devices.magnet.heater_control = 'off'

In [45]:
devices.magnet.heater_control = 'on'

In [46]:
devices.magnet.heater_control = 'force'

In [ ]:
# only run if magnet is at zero
# devices.magnet.activity_control = "clamp"

In [47]:
devices.magnet.activity_control = "hold"

In [48]:
devices.magnet.activity_control = "to_zero"

In [49]:
devices.magnet.activity_control = "to_set_point"

### list all properties

In [64]:
for prop in devices.magnet.get_pyscan_properties():
    if 'write_only' not in devices.magnet[f'_{prop}_settings']:
        print(f'{prop}: {devices.magnet[prop]}')

field_set_point: -5.0
field_rate: 0.2
current_set_point: -22.4562
current_rate: 0.898
output_current: -22.4562
voltage: -0.43
measured_current: -22.27
output_field: -5.0
software_voltage_limit: 2.5
persistent_current: -19.5229
trip_field: -32.8032
persistent_field: -4.34687
switch_heater_current: 20.0
safe_current_limit_negative: -40.42
safe_current_limit_positive: 40.42
lead_resistance: 18.6
magnet_inductance: 51.7
firmware_version: IPS120-10  Version 3.07  (c) OXFORD 1996
status_string: X00A1C3H1M10P71


### methods

In [50]:
devices.magnet.local() # keyword: locked=False

In [51]:
devices.magnet.remote() # keyword: locked=False

In [ ]:
devices.magnet.heater('off')

In [ ]:
devices.magnet.heater('on')

In [52]:
devices.magnet.to_set_point()

In [55]:
devices.magnet.hold()

In [54]:
devices.magnet.to_zero()

In [56]:
status = devices.magnet.status()
print(status)


system status (operation) : Normal (X1=0)
system status (voltage) : Normal (X2=0)
Activity ; Hold (A=0)
LOC/REM status ; Remote & Unlocked (C=3)
Heater ; On (H=1)
Mode (rate) ; Tesla, Immediate, Fast (M1=1)
Mode (sweep) ; At Rest (M2=0)


### magnet field property - only test with magnet cold

In [57]:
devices.magnet.hold()
devices.magnet.remote()
devices.magnet.heater('on')
original_rate = devices.magnet.field_rate
devices.magnet.field_rate = 0.1
value = devices.magnet.output_field
db = 0.01
new_value = value + db
if new_value > devices.magnet.field_limit:
    new_value = value - db
devices.magnet.field = new_value
assert devices.magnet.output_field == new_value

In [58]:
devices.magnet.field = value
assert devices.magnet.output_field == value

In [59]:
devices.field_rate = original_rate

In [ ]:
devices.magnet.heater("off")

### go to a field

In [60]:
devices.magnet.hold()
devices.magnet.heater('on')
devices.magnet.field_set_point = -5
devices.magnet.field_rate = 0.2
devices.magnet.to_set_point()

## help

In [ ]:
devices.magnet?

In [ ]:
devices.magnet.output_field?